<a href="https://colab.research.google.com/github/fabriziobasso/kaggle/blob/main/File_00_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<h1 align="center"><font color='#001ddd'> GLUCOSE PREDICTION DATASET**</font></h1>

## **Dataset Description**
The dataset is from a study that collected data from young adults in the UK with type 1 diabetes, who used a continuous glucose monitor (CGM), an insulin pump and a smartwatch. These devices collected blood glucose readings, insulin dosage, carbohydrate intake, and activity data. The data collected was aggregated to five-minute intervals and formatted into samples. Each sample represents a point in time and includes the aggregated five-minute intervals from the previous six hours. The aim is to predict the blood glucose reading an hour into the future, for each of these samples.

The training set takes samples from the first three months of study data from nine of the participants and includes the future blood glucose value. These training samples appear in chronological order and overlap. The testing set takes samples from the remainder of the study period from fifteen of the participants (so unseen participants appear in the testing set). These testing samples do not overlap and are in a random order to avoid data leakage.

**Complexities to be aware of:**

This is medical data so there are missing values and noise in the data
the participants did not all use the same device models (CGM, insulin pump and smartwatch) so there may be differences in the collection method of the data
some participants in the test set do not appear in the training set

In [1]:
%%capture
# Connect to Colab:#
from google.colab import drive
import os
drive.mount('/content/drive')

!pip install category-encoders
!pip install optuna
!pip install optuna-integration
#!pip install scikit-learn==1.4
!pip install catboost
!pip install deeptables

!pip install keras-tuner --upgrade
!pip install keras-nlp
!pip install BorutaShap
!pip install scikit-lego
!!pip install --no-index -U --find-links=/kaggle/input/deeptables-v0-2-5/deeptables-0.2.5 deeptables==0.2.5

In [2]:
folder_script = models_folders = "/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose"
os.chdir(folder_script)

In [3]:
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.wrapper import PolynomialWrapper
from category_encoders.count import CountEncoder

# Setup notebook
from pathlib import Path
import ipywidgets as widgets
import pandas as pd
import numpy as np
from pickle import load, dump
import json
import joblib
#import calplot as cal
import missingno as msno
import category_encoders as ce

# Graphic Libraries:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

# Palette Setup
colors = ['#FB5B68','#FFEB48','#2676A1','#FFBDB0',]
colormap_0 = mpl.colors.LinearSegmentedColormap.from_list("",colors)
palette_1 = sns.color_palette("coolwarm", as_cmap=True)
palette_2 = sns.color_palette("YlOrBr", as_cmap=True)
palette_3 = sns.light_palette("red", as_cmap=True)
palette_4 = sns.color_palette("viridis", as_cmap=True)
palette_5 = sns.color_palette("rocket", as_cmap=True)
palette_6 = sns.color_palette("GnBu", as_cmap=True)
palette_7 = sns.color_palette("tab20c", as_cmap=False)
palette_8 = sns.color_palette("Set2", as_cmap=False)

palette_custom = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc','#e5d8bd','#fddaec','#f2f2f2']
palette_9 = sns.color_palette(palette_custom, as_cmap=False)


# Bloomberg
#from xbbg import blp
from catboost import CatBoostRegressor, Pool
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from xgboost.callback import EarlyStopping

import lightgbm as lgb
from lightgbm import (LGBMRegressor,
                      LGBMClassifier,
                      early_stopping,
                      record_evaluation,
                      log_evaluation)

# Time Management
from tqdm import tqdm
from datetime import date
from datetime import datetime
from pandas.tseries.offsets import BMonthEnd, QuarterEnd
import datetime
from pandas.tseries.offsets import BDay # BDay is business day, not birthday...
import datetime as dt
import click
import glob
import os
import gc
import re
import string

from ipywidgets import AppLayout
from ipywidgets import Dropdown, Layout, HTML, AppLayout, VBox, Label, HBox, BoundedFloatText, interact, Output

#from my_func import *

import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import keras
from keras import ops
from keras import layers

from keras.layers import Input, LSTM, Dense, Lambda, RepeatVector, Reshape
from keras.models import Model
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

from keras.utils import FeatureSpace, plot_model

# Import libraries for Hypertuning
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch, GridSearch, BayesianOptimization

#from my_func import *

# preprocessing modules
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, cross_validate, GroupKFold, GridSearchCV, RepeatedStratifiedKFold, cross_val_predict

from sklearn.preprocessing import (LabelEncoder,
                                   StandardScaler,
                                   MinMaxScaler,
                                   OrdinalEncoder,
                                   RobustScaler,
                                   PowerTransformer,
                                   OneHotEncoder,
                                   LabelEncoder,
                                   QuantileTransformer,
                                   PolynomialFeatures)

# metrics
import sklearn
from sklearn.metrics import (mean_squared_error,
                             root_mean_squared_error,
                             r2_score,
                             mean_absolute_error,
                             mean_absolute_percentage_error,
                             classification_report,
                             confusion_matrix,
                             ConfusionMatrixDisplay,
                             multilabel_confusion_matrix,
                             accuracy_score,
                             roc_auc_score,
                             auc,
                             roc_curve,
                             log_loss,
                             make_scorer)

# modeling algos
from sklearn.linear_model import (LogisticRegression,
                                  Lasso,
                                  ridge_regression,
                                  LinearRegression,
                                  Ridge,
                                  RidgeCV,
                                  ElasticNet,
                                  BayesianRidge,
                                  HuberRegressor,
                                  TweedieRegressor,
                                  QuantileRegressor,
                                  ARDRegression,
                                  TheilSenRegressor,
                                  PoissonRegressor,
                                  GammaRegressor)

from sklearn.ensemble import (AdaBoostRegressor,
                              AdaBoostClassifier,
                              RandomForestRegressor,
                              RandomForestClassifier,
                              VotingRegressor,
                              GradientBoostingRegressor,
                              GradientBoostingClassifier,
                              StackingRegressor,
                              HistGradientBoostingClassifier,
                              HistGradientBoostingRegressor,
                              ExtraTreesClassifier)

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

import seaborn as sns
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

import itertools
import warnings
import logging
from openpyxl import load_workbook

import statsmodels.api as sm
from pylab import rcParams
import scipy.stats as ss

warnings.filterwarnings('ignore')
#plt.style.use('fivethirtyeight')

# Setting rc parameters in seaborn for plots and graphs-
# Reference - https://matplotlib.org/stable/tutorials/introductory/customizing.html:-
# To alter this, refer to matplotlib.rcParams.keys()

sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.5,
         'grid.alpha'           :0.5,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : 'black',
         'axes.titlesize'       : 12,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        });

sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5})
# Set Style
mpl.rcParams['figure.dpi'] = 120;

# Making sklearn pipeline outputs as dataframe:-
pd.set_option('display.max_columns', 100);
pd.set_option('display.max_rows', 50);

sns.despine(left=True, bottom=True, top=False, right=False)

mpl.rcParams['axes.spines.left'] = True
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = True

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


<Figure size 960x660 with 0 Axes>

## 1.0 Functions:

In [4]:
def encode_target(y_train, y_test, encoder_type='label', enc_strategy=False):
    """
    Encodes the target columns in the training and testing data
    using the specified encoder type.

    Parameters:
    y_train (pd.Series or pd.DataFrame): Training target data.
    y_test (pd.Series or pd.DataFrame): Testing target data.

    Returns:
    y_train_encoded (pd.Series): Encoded training target data.
    y_test_encoded (pd.Series): Encoded testing target data.
    """

    if encoder_type == 'label':
        encoder = LabelEncoder()
        y_train_encoded = encoder.fit_transform(y_train)
        y_test_encoded = encoder.transform(y_test)

        y_train_encoded = pd.Series(y_train_encoded, index=y_train.index, name="Target")
        y_test_encoded = pd.Series(y_test_encoded, index=y_test.index, name="Target")


    elif encoder_type == 'onehot':
        y_train_ = y_train.values.reshape(-1, 1)
        y_test_ = y_test.values.reshape(-1, 1)

        encoder = OneHotEncoder(sparse_output=False)
        y_train_encoded = encoder.fit_transform(y_train_)
        y_test_encoded = encoder.transform(y_test_)

        y_train_encoded = pd.DataFrame(y_train_encoded, index=y_train.index)
        y_test_encoded = pd.DataFrame(y_test_encoded, index=y_test.index)

    else:
        raise ValueError("Invalid encoder_type. Currently supported: 'label'.")

    if enc_strategy:
        return y_train_encoded, y_test_encoded, encoder

    else:
        return y_train_encoded, y_test_encoded

def encode_data(X_train, X_test, encoder_type='label', columns=None, map=None):
    """
    Encodes the training and testing data using the specified encoder type.

    Parameters:
    X_train (pd.DataFrame): Training data.
    X_test (pd.DataFrame): Testing data.
    encoder_type (str): Type of encoder ('label' or 'onehot'). Default is 'label'.
    columns (list): List of columns to encode. If None, all object type columns are encoded.

    Returns:
    X_train_encoded (pd.DataFrame): Encoded training data.
    X_test_encoded (pd.DataFrame): Encoded testing data.
    """

    if columns is None:
        # Default to all object type columns if no columns are specified
        columns = X_train.select_dtypes(include=['object']).columns.tolist()

    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()

    if encoder_type == 'label':
        for col in columns:
            le = LabelEncoder()
            X_train_encoded[col] = le.fit_transform(X_train[col])
            X_test_encoded[col] = le.transform(X_test[col])

    elif encoder_type == 'onehot':
        for col in columns:
            ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')
            # Fit the encoder on the training data and transform both training and test data
            encoded_train = ohe.fit_transform(X_train[[col]])
            encoded_test = ohe.transform(X_test[[col]])

            # Create a DataFrame with the encoded data
            encoded_train_df = pd.DataFrame(encoded_train, columns=ohe.get_feature_names_out([col]))
            encoded_test_df = pd.DataFrame(encoded_test, columns=ohe.get_feature_names_out([col]))

            # Concatenate the new columns to the original dataframes and drop the original columns
            X_train_encoded = pd.concat([X_train_encoded.drop(col, axis=1), encoded_train_df], axis=1)
            X_test_encoded = pd.concat([X_test_encoded.drop(col, axis=1), encoded_test_df], axis=1)

    elif encoder_type == 'count_encoder':

          for col in columns:

                target_encoder = CountEncoder(cols=columns)
                X_train_encoded = target_encoder.fit_transform(X_train_encoded)
                X_test_encoded = target_encoder.transform(X_test_encoded)

    else:
        raise ValueError("Invalid encoder_type. Currently supported: 'label', 'onehot', 'target_encoder'.")

    return X_train_encoded, X_test_encoded

## **Importing the Dataset**

## **Files**
* activities.txt - a list of activity names that appear in the activity-X:XX columns
* sample_submission.csv - a sample submission file in the correct format
* test.csv - the test set
* train.csv - the training set

## **Columns**
* train.csv:
    * **id - row id** consisting of participant number and a count for that participant
    * **p_num** - participant number
    * **time** - time of day in the format HH:MM:SS
    * **bg-X:XX** - blood glucose reading in mmol/L, X:XX(H:SS) time in the past (e.g. bg-2:35, would be the blood glucose reading from 2 hours and 35 minutes before the time value for that row), recorded by the continuous glucose monitor
    * **insulin-X:XX** - total insulin dose received in units in the last 5 minutes, X:XX(H:SS) time in the past (e.g. insulin-2:35, would be the total insulin dose received between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the insulin pump
    * **carbs-X:XX** - total carbohydrate value consumed in grammes in the last 5 minutes, X:XX(H:SS) time in the past (e.g. carbs-2:35, would be the total carbohydrate value consumed between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the participant
    * **hr-X:XX** - mean heart rate in beats per minute in the last 5 minutes, X:XX(H:SS) time in the past (e.g. hr-2:35, would be the mean heart rate between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the smartwatch
    * **steps-X:XX** - total steps walked in the last 5 minutes, X:XX(H:SS) time in the past (e.g. * steps-2:35, would be the total steps walked between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the smartwatch
    * **cals-X:XX** - total calories burnt in the last 5 minutes, X:XX(H:SS) time in the past (e.g. cals-2:35, would be the total calories burned between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), calculated by the smartwatch
    * **activity-X:XX** - self-declared activity performed in the last 5 minutes, X:XX(H:SS) time in the past (e.g. activity-2:35, would show a string name of the activity performed between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), set on the smartwatch
    * **bg+1:00** - blood glucose reading in mmol/L an hour in the future, this is the value you will be predicting (not provided in test.csv)

In [5]:
df=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/train.csv", index_col=0)
df_test=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/test.csv", index_col=0)

df.shape

(177024, 507)

In [6]:
df.p_num.unique()

array(['p01', 'p02', 'p03', 'p04', 'p05', 'p06', 'p10', 'p11', 'p12'],
      dtype=object)

In [7]:
df.head(5)

,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,bg-5:15,bg-5:10,bg-5:05,bg-5:00,bg-4:55,bg-4:50,bg-4:45,bg-4:40,bg-4:35,bg-4:30,bg-4:25,bg-4:20,bg-4:15,bg-4:10,bg-4:05,bg-4:00,bg-3:55,bg-3:50,bg-3:45,bg-3:40,bg-3:35,bg-3:30,bg-3:25,bg-3:20,bg-3:15,bg-3:10,bg-3:05,bg-3:00,bg-2:55,bg-2:50,bg-2:45,bg-2:40,bg-2:35,bg-2:30,bg-2:25,bg-2:20,bg-2:15,bg-2:10,bg-2:05,bg-2:00,...,activity-4:00,activity-3:55,activity-3:50,activity-3:45,activity-3:40,activity-3:35,activity-3:30,activity-3:25,activity-3:20,activity-3:15,activity-3:10,activity-3:05,activity-3:00,activity-2:55,activity-2:50,activity-2:45,activity-2:40,activity-2:35,activity-2:30,activity-2:25,activity-2:20,activity-2:15,activity-2:10,activity-2:05,activity-2:00,activity-1:55,activity-1:50,activity-1:45,activity-1:40,activity-1:35,activity-1:30,activity-1:25,activity-1:20,activity-1:15,activity-1:10,activity-1:05,activity-1:00,activity-0:55,activity-0:50,activity-0:45,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,NaN,9.2,NaN,NaN,8.7,NaN,NaN,8.4,NaN,NaN,8.1,NaN,NaN,8.3,NaN,NaN,9.6,NaN,NaN,11.1,NaN,NaN,11.8,NaN,NaN,12.8,NaN,NaN,13.9,NaN,NaN,14.2,NaN,NaN,14.2,NaN,NaN,15.4,NaN,NaN,17.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,NaN,8.7,NaN,NaN,8.4,NaN,NaN,8.1,NaN,NaN,8.3,NaN,NaN,9.6,NaN,NaN,11.1,NaN,NaN,11.8,NaN,NaN,12.8,NaN,NaN,13.9,NaN,NaN,14.2,NaN,NaN,14.2,NaN,NaN,15.4,NaN,NaN,17.2,NaN,NaN,18.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,NaN,8.4,NaN,NaN,8.1,NaN,NaN,8.3,NaN,NaN,9.6,NaN,NaN,11.1,NaN,NaN,11.8,NaN,NaN,12.8,NaN,NaN,13.9,NaN,NaN,14.2,NaN,NaN,14.2,NaN,NaN,15.4,NaN,NaN,17.2,NaN,NaN,18.2,NaN,NaN,18.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,NaN,8.1,NaN,NaN,8.3,NaN,NaN,9.6,NaN,NaN,11.1,NaN,NaN,11.8,NaN,NaN,12.8,NaN,NaN,13.9,NaN,NaN,14.2,NaN,NaN,14.2,NaN,NaN,15.4,NaN,NaN,17.2,NaN,NaN,18.2,NaN,NaN,18.4,NaN,NaN,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,NaN,8.3,NaN,NaN,9.6,NaN,NaN,11.1,NaN,NaN,11.8,NaN,NaN,12.8,NaN,NaN,13.9,NaN,NaN,14.2,NaN,NaN,14.2,NaN,NaN,15.4,NaN,NaN,17.2,NaN,NaN,18.2,NaN,NaN,18.4,NaN,NaN,18.0,NaN,NaN,17.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


In [8]:
df_test.head()

,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,bg-5:15,bg-5:10,bg-5:05,bg-5:00,bg-4:55,bg-4:50,bg-4:45,bg-4:40,bg-4:35,bg-4:30,bg-4:25,bg-4:20,bg-4:15,bg-4:10,bg-4:05,bg-4:00,bg-3:55,bg-3:50,bg-3:45,bg-3:40,bg-3:35,bg-3:30,bg-3:25,bg-3:20,bg-3:15,bg-3:10,bg-3:05,bg-3:00,bg-2:55,bg-2:50,bg-2:45,bg-2:40,bg-2:35,bg-2:30,bg-2:25,bg-2:20,bg-2:15,bg-2:10,bg-2:05,bg-2:00,...,activity-4:05,activity-4:00,activity-3:55,activity-3:50,activity-3:45,activity-3:40,activity-3:35,activity-3:30,activity-3:25,activity-3:20,activity-3:15,activity-3:10,activity-3:05,activity-3:00,activity-2:55,activity-2:50,activity-2:45,activity-2:40,activity-2:35,activity-2:30,activity-2:25,activity-2:20,activity-2:15,activity-2:10,activity-2:05,activity-2:00,activity-1:55,activity-1:50,activity-1:45,activity-1:40,activity-1:35,activity-1:30,activity-1:25,activity-1:20,activity-1:15,activity-1:10,activity-1:05,activity-1:00,activity-0:55,activity-0:50,activity-0:45,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
p01_8459,p01,06:45:00,NaN,9.2,NaN,NaN,10.2,NaN,NaN,10.3,NaN,NaN,10.2,NaN,NaN,11.7,NaN,NaN,13.5,NaN,NaN,15.3,NaN,NaN,15.0,NaN,NaN,14.4,NaN,NaN,13.6,NaN,NaN,12.6,NaN,NaN,11.9,NaN,NaN,11.4,NaN,NaN,11.9,NaN,NaN,12.9,NaN,NaN,13.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p01_8460,p01,11:25:00,NaN,NaN,9.9,NaN,NaN,9.4,NaN,NaN,9.1,NaN,NaN,8.3,NaN,NaN,7.7,NaN,NaN,7.8,NaN,NaN,7.7,NaN,NaN,7.1,NaN,NaN,7.1,NaN,NaN,6.7,NaN,NaN,6.6,NaN,NaN,6.9,NaN,NaN,8.6,NaN,NaN,9.7,NaN,NaN,9.1,NaN,NaN,7.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walk,Walk,Walk
p01_8461,p01,14:45:00,NaN,5.5,NaN,NaN,5.5,NaN,NaN,5.2,NaN,NaN,5.2,NaN,NaN,NaN,5.1,NaN,NaN,5.1,NaN,NaN,4.7,NaN,NaN,4.4,NaN,NaN,4.8,NaN,NaN,6.1,NaN,NaN,6.9,NaN,NaN,6.3,NaN,NaN,5.2,NaN,NaN,4.7,NaN,NaN,5.1,NaN,NaN,5.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p01_8462,p01,04:30:00,NaN,3.4,NaN,NaN,3.9,NaN,NaN,4.7,NaN,NaN,4.1,NaN,NaN,3.5,NaN,NaN,3.9,NaN,NaN,4.3,NaN,NaN,4.6,NaN,NaN,3.8,NaN,NaN,3.1,NaN,NaN,3.6,NaN,NaN,4.2,NaN,NaN,4.2,NaN,NaN,4.1,NaN,NaN,4.2,NaN,NaN,3.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p01_8463,p01,04:20:00,NaN,NaN,8.3,NaN,NaN,10.0,NaN,NaN,12.2,NaN,NaN,12.8,NaN,NaN,12.8,NaN,NaN,12.2,NaN,NaN,11.4,NaN,NaN,12.1,NaN,NaN,11.8,NaN,NaN,10.7,NaN,NaN,9.6,NaN,NaN,8.9,NaN,NaN,8.4,NaN,NaN,7.3,NaN,NaN,5.8,NaN,NaN,4.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_all = pd.concat([df,df_test])

Sub-dataset are created for each main set of features to inpute missing values:

In [69]:
# Select columns containing the word "bg"
bg_col = df_all.filter(regex='bg|time|p_num')
insulin_col = df_all.filter(regex='insulin|time|p_num')
carb_col = df_all.filter(regex='carbs|time|p_num')
hr_col = df_all.filter(regex='hr|time|p_num')

## **1.0 - Fill bg-xx:xx NaN**

In [11]:
bg_col_T = bg_col.drop(columns=["p_num","time","bg+1:00"]).T.copy()
bg_col_T.interpolate(inplace=True)
bg_col_T.ffill(inplace=True)
bg_col_T.bfill(inplace=True)

bg_col_T.isna().sum().sum()

0

In [12]:
df_all[bg_col_T.T.columns] = np.round(bg_col_T.T,1).values

## **2.0 - Fill Insulin NaN**

In [13]:
insulin_col.isna().sum().sum()

688995

In [ ]:
#msno.matrix(insulin_col,figsize=(12,5), fontsize=10);

**The operation below is repeated three times. For:"p11","p15","p21"**

**p11**

In [15]:
insulin_fillna = insulin_col[insulin_col["p_num"]=="p11"].drop(["p_num"],axis=1)#.reset_index()#.mean(axis=1)
print(insulin_fillna.shape)
display(insulin_fillna.head(),insulin_fillna.isna().sum().sum())

(24776, 73)


,time,insulin-5:55,insulin-5:50,insulin-5:45,insulin-5:40,insulin-5:35,insulin-5:30,insulin-5:25,insulin-5:20,insulin-5:15,insulin-5:10,insulin-5:05,insulin-5:00,insulin-4:55,insulin-4:50,insulin-4:45,insulin-4:40,insulin-4:35,insulin-4:30,insulin-4:25,insulin-4:20,insulin-4:15,insulin-4:10,insulin-4:05,insulin-4:00,insulin-3:55,insulin-3:50,insulin-3:45,insulin-3:40,insulin-3:35,insulin-3:30,insulin-3:25,insulin-3:20,insulin-3:15,insulin-3:10,insulin-3:05,insulin-3:00,insulin-2:55,insulin-2:50,insulin-2:45,insulin-2:40,insulin-2:35,insulin-2:30,insulin-2:25,insulin-2:20,insulin-2:15,insulin-2:10,insulin-2:05,insulin-2:00,insulin-1:55,insulin-1:50,insulin-1:45,insulin-1:40,insulin-1:35,insulin-1:30,insulin-1:25,insulin-1:20,insulin-1:15,insulin-1:10,insulin-1:05,insulin-1:00,insulin-0:55,insulin-0:50,insulin-0:45,insulin-0:40,insulin-0:35,insulin-0:30,insulin-0:25,insulin-0:20,insulin-0:15,insulin-0:10,insulin-0:05,insulin-0:00
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
p11_0,06:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p11_1,06:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p11_2,06:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p11_3,06:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p11_4,06:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


677556

In [16]:
unique_times = insulin_fillna.time.unique()

for time in tqdm(unique_times):
    list_of_means = insulin_fillna[insulin_fillna.time==time].iloc[:,1:].mean(axis=0)

    for col in insulin_fillna.columns[1:]:

      mask = (insulin_fillna['time'] == time)
      insulin_fillna.loc[mask, col] = insulin_fillna.loc[mask, col].fillna(list_of_means[col])

100%|██████████| 288/288 [01:24<00:00,  3.43it/s]


In [17]:
insulin_fillna.head()

,time,insulin-5:55,insulin-5:50,insulin-5:45,insulin-5:40,insulin-5:35,insulin-5:30,insulin-5:25,insulin-5:20,insulin-5:15,insulin-5:10,insulin-5:05,insulin-5:00,insulin-4:55,insulin-4:50,insulin-4:45,insulin-4:40,insulin-4:35,insulin-4:30,insulin-4:25,insulin-4:20,insulin-4:15,insulin-4:10,insulin-4:05,insulin-4:00,insulin-3:55,insulin-3:50,insulin-3:45,insulin-3:40,insulin-3:35,insulin-3:30,insulin-3:25,insulin-3:20,insulin-3:15,insulin-3:10,insulin-3:05,insulin-3:00,insulin-2:55,insulin-2:50,insulin-2:45,insulin-2:40,insulin-2:35,insulin-2:30,insulin-2:25,insulin-2:20,insulin-2:15,insulin-2:10,insulin-2:05,insulin-2:00,insulin-1:55,insulin-1:50,insulin-1:45,insulin-1:40,insulin-1:35,insulin-1:30,insulin-1:25,insulin-1:20,insulin-1:15,insulin-1:10,insulin-1:05,insulin-1:00,insulin-0:55,insulin-0:50,insulin-0:45,insulin-0:40,insulin-0:35,insulin-0:30,insulin-0:25,insulin-0:20,insulin-0:15,insulin-0:10,insulin-0:05,insulin-0:00
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
p11_0,06:05:00,0.080842,0.098657,0.080072,0.136675,0.080072,0.078766,0.076809,0.107706,0.236604,0.075000,0.103302,0.075000,0.075000,0.075000,0.075708,0.097642,0.075708,0.075000,0.075000,0.275000,0.075000,0.075000,0.200236,0.075000,0.075708,0.127830,0.075000,0.124057,0.075000,0.075000,0.246698,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.182547,0.075000,0.075000,0.075000,0.075000,0.075000,0.091700,0.127549,0.157738,0.138870,0.091700,0.091700,0.091700,0.091700,0.205928,0.090834,0.128570,0.090834,0.090834,0.156872,0.090834,0.090834,0.090834,0.089968,0.146572,0.108836,0.089968,0.116383,0.089968,0.089968,0.073585
p11_1,06:10:00,0.101689,0.086689,0.143218,0.087862,0.086047,0.144715,0.112095,0.285876,0.077553,0.102273,0.075000,0.075000,0.075000,0.075682,0.096818,0.075682,0.075418,0.076231,0.269380,0.076971,0.077207,0.198060,0.077000,0.076880,0.126978,0.076436,0.124496,0.077973,0.078395,0.243971,0.078000,0.076907,0.075987,0.076153,0.076896,0.077625,0.078275,0.079040,0.078635,0.078253,0.181829,0.078287,0.077785,0.077133,0.077295,0.077278,0.094498,0.128727,0.156931,0.138980,0.093013,0.092973,0.093025,0.092549,0.202427,0.092904,0.128671,0.090940,0.091167,0.154502,0.090860,0.090662,0.089836,0.088513,0.142909,0.106545,0.088364,0.113715,0.087884,0.088098,0.072500,0.072335
p11_2,06:15:00,0.080072,0.136675,0.080072,0.078766,0.076809,0.107706,0.236604,0.075000,0.103302,0.075000,0.075000,0.075000,0.075708,0.097642,0.075708,0.075000,0.075000,0.275000,0.075000,0.075000,0.200236,0.075000,0.075708,0.127830,0.075000,0.124057,0.075000,0.075000,0.246698,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.182547,0.075000,0.075000,0.075000,0.075000,0.075000,0.091700,0.127549,0.157738,0.138870,0.091700,0.091700,0.091700,0.091700,0.205928,0.090834,0.128570,0.090834,0.090834,0.156872,0.090834,0.090834,0.090834,0.089968,0.146572,0.108836,0.089968,0.116383,0.089968,0.089968,0.073585,0.073585,0.160377
p11_3,06:20:00,0.135533,0.079978,0.078696,0.076776,0.107100,0.233611,0.075000,0.102778,0.074998,0.074843,0.074485,0.074913,0.096372,0.074902,0.074143,0.074089,0.270533,0.074430,0.074641,0.197798,0.075000,0.076035,0.128026,0.076826,0.125276,0.077143,0.076861,0.244930,0.076100,0.075806,0.075489,0.075248,0.075000,0.075000,0.075000,0.075250,0.075961,0.076359,0.181337,0.075085,0.075304,0.076528,0.077926,0.077635,0.092559,0.126983,0.157033,0.139843,0.094319,0.093709,0.092735,0.091972,0.203837,0.090863,0.127833,0.090798,0.090804,0.155639,0.090850,0.090850,0.090848,0.089998,0.145554,0.108515,0.089976,0.115639,0.089041,0.088559,0.072567,0.072789,0.157944,0.152407
p11_4,06:25:00,0.080072,0.078766,0.076809,0.107706,0.236604,0.075000,0.103302,0.075000,0.075000,0.075000,0.075708,0.097642,0.075708,0.075000,0.075000,0.275000,0.075000,0.075000,0.200236,0.075000,0.075708,0.127830,0.075000,0.124057,0.075000,0.075000,0.246698,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,0.075000,

In [18]:
#df_all.filter(regex='insulin|time|id|p_num')
insulin_fillna.isna().sum().sum(),insulin_fillna.shape,insulin_col.shape

(0, (24776, 73), (180668, 74))

**Replace values in df**

In [19]:
insulin_col.loc[insulin_fillna.index, insulin_fillna.columns] = insulin_fillna.values
insulin_col.loc[insulin_fillna.index, insulin_fillna.columns].isna().sum().sum(),insulin_col.shape

(0, (180668, 74))

In [20]:
insulin_col[insulin_col["insulin-5:55"].isna()==True].p_num.unique()

array(['p15', 'p21'], dtype=object)

In [ ]:
#msno.matrix(insulin_col,figsize=(12,5), fontsize=10);

**-----------------------------p15-----------------------------**

In [22]:
p15 = insulin_col[insulin_col["p_num"]=="p15"].copy()
p15_sorted = insulin_col[insulin_col["p_num"]=="p15"].sort_values(by="time").copy()

In [23]:
#display(p15.head())
#display(p15_sorted.head())

In [24]:
p15_sorted.bfill(inplace=True)
p15_sorted.ffill(inplace=True)
p15_sorted.isna().sum().sum()

0

In [25]:
#p15_sorted = p15_sorted.loc[p15.index]
#p15_sorted

In [26]:
insulin_col.loc[p15_sorted.index, p15_sorted.columns] = p15_sorted.values
insulin_col.loc[p15_sorted.index, p15_sorted.columns].isna().sum().sum()

0

In [27]:
insulin_col[insulin_col["insulin-0:00"].isna()==True].p_num.unique()

array(['p06', 'p12', 'p21'], dtype=object)

In [28]:
df_all[insulin_col.columns] = insulin_col.values

**-----------------------------p21-----------------------------**

In [29]:
p21 = insulin_col[insulin_col["p_num"]=="p21"].copy()
p21_sorted = insulin_col[insulin_col["p_num"]=="p21"].sort_values(by="time").copy()

In [30]:
p21[p21["insulin-0:00"].isna()==True].index#.sum().sum()

Index(['p21_11', 'p21_66', 'p21_80', 'p21_200', 'p21_225'], dtype='object', name='id')

In [31]:
p21.loc["p21_11"] = p21.loc["p21_11"].fillna(method="ffill").values
p21.loc["p21_66","insulin-5:55":] = p21_sorted.iloc[207:212,2:].mean(axis=0).values
p21.loc["p21_80","insulin-5:55":] = p21_sorted.iloc[65:74,2:].mean(axis=0).values
p21.loc["p21_200","insulin-5:55":] = p21_sorted.iloc[74:83,2:].mean(axis=0).values
p21.loc["p21_225","insulin-5:55":] = p21_sorted.iloc[166:177,2:].mean(axis=0).values

In [32]:
#p21_sorted.iloc[166:177]#.mean(axis=0)
#p21_sorted.loc[:"p21_225"]

In [33]:
(insulin_col.index==insulin_col.index).all()

True

In [34]:
insulin_col.loc[p21.index, p21.columns] = p21.values
insulin_col.isna().sum().sum()

112

**-----------------------------p06-----------------------------**

In [35]:
p06 = insulin_col[insulin_col["p_num"]=="p06"].copy()
p06_sorted = insulin_col[insulin_col["p_num"]=="p06"].sort_values(by="time").copy()

In [36]:
p06[p06["insulin-0:00"].isna()==True].index#.sum().sum()

Index(['p06_8396'], dtype='object', name='id')

In [37]:
#p06_sorted.iloc[1670:1684]#.mean(axis=0)
#p06_sorted.loc[:"p06_8396"]

In [38]:
p06.loc["p06_8396","insulin-5:25":] = p06_sorted.iloc[1670:1684,8:].mean(axis=0).values

In [39]:
p06[p06["insulin-0:00"].isna()==True].index#.sum().sum()

Index([], dtype='object', name='id')

In [40]:
insulin_col.loc[p06.index, p06.columns] = p06.values
insulin_col.isna().sum().sum()

46

**-----------------------------p12-----------------------------**

In [41]:
p12 = insulin_col[insulin_col["p_num"]=="p12"].copy()
p12_sorted = insulin_col[insulin_col["p_num"]=="p12"].sort_values(by="time").copy()

In [42]:
p12[p12["insulin-0:00"].isna()==True].index#.sum().sum()

Index(['p12_25319'], dtype='object', name='id')

In [43]:
#p12_sorted.iloc[1670:1684]#.mean(axis=0)
#p12_sorted[p12_sorted.time=="21:00:00"]#.loc[:"p12_25319"]

In [44]:
p12.loc["p12_25319","insulin-3:45":] = p12_sorted[p12_sorted.time=="21:00:00"].iloc[:,28:].mean(axis=0).values

In [45]:
p12[p12["insulin-0:00"].isna()==True].index#.sum().sum()

Index([], dtype='object', name='id')

In [46]:
insulin_col.loc[p12.index, p12.columns] = p12.values
insulin_col.isna().sum().sum()

0

### Replace in original dataset:

In [47]:
df_all[insulin_col.columns] = insulin_col.values

In [48]:
insulin_col = df_all.filter(regex='insulin|time|id|p_num')
insulin_col.isna().sum().sum()

0

## **3.0 - Fill NaN for Carb Columns**

In [67]:
#carb_col_ = carb_col.fillna(0)
#carb_col_.iloc[28:50,2:].T.plot()

The most reasonable way to fill the missing values for these features seem to be with 0s.

In [65]:
df_all[df_all.filter(regex='carbs').columns] = df_all.filter(regex='carbs').fillna(0)

## **4.0 - Fill NaN for HR Columns**

In [ ]:
#msno.matrix(hr_col,figsize=(12,5), fontsize=10);

In [84]:
#hr_col.reset_index().to_csv("hr.csv")

In [108]:
df = hr_col.copy()

# Get all column names
all_columns = df.columns

# Select the columns that start with 'hr-'
columns_to_fill = [col for col in all_columns if col.startswith('hr-')]

# Group by 'p_num' and 'time', calculate the mean for each group
grouped_df = df.groupby(['p_num', 'time'])[columns_to_fill].mean().reset_index()

# Merge the grouped DataFrame with the original DataFrame
merged_df = pd.merge(df, grouped_df, on=['p_num', 'time'], how='left', suffixes=('', '_mean'))

# Select columns in `columns_to_fill` from `merged_df`
filled_df = merged_df[columns_to_fill]

# Iterate over columns and fill missing values
for col in columns_to_fill:
    mean_col = col + '_mean'  # Get the name of the corresponding mean column
    filled_df[col] = filled_df[col].fillna(merged_df[mean_col])

filled_df = filled_df.ffill(axis=1)
filled_df = filled_df.bfill(axis=1)

In [109]:
hr_col[filled_df.columns] = filled_df.values

In [112]:
df = hr_col.copy()

# Get all column names
all_columns = df.columns

# Select the columns that start with 'hr-'
columns_to_fill = [col for col in all_columns if col.startswith('hr-')]

# Group by 'p_num' and 'time', calculate the mean for each group
grouped_df = df.groupby(['p_num'])[columns_to_fill].mean().reset_index()

# Merge the grouped DataFrame with the original DataFrame
merged_df = pd.merge(df, grouped_df, on=['p_num'], how='left', suffixes=('', '_mean'))

# Select columns in `columns_to_fill` from `merged_df`
filled_df = merged_df[columns_to_fill]

# Iterate over columns and fill missing values
for col in columns_to_fill:
    mean_col = col + '_mean'  # Get the name of the corresponding mean column
    filled_df[col] = filled_df[col].fillna(merged_df[mean_col])

In [116]:
filled_df = np.round(filled_df,1)
hr_col[filled_df.columns] = filled_df.values

In [ ]:
#msno.matrix(hr_col,figsize=(12,5), fontsize=10);

### Replace in original dataset:

In [118]:
df_all[hr_col.columns] = hr_col.values

In [119]:
hr_col = df_all.filter(regex='hr|time|id|p_num')
hr_col.isna().sum().sum()

0

In [ ]:
df_all.reset_index(inplace=True)
df_all.to_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/df_all_filled.csv", index=False)
df_all.head()

In [ ]:
df_all.tail()

In [ ]:
insulin_col = df_all.filter(regex='insulin|time|id|p_num')
insulin_col.isna().sum().sum()

0